In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [30]:
df = pd.read_csv("data_ds.csv")
target = "winner"

# drop useless info
df.drop(['R_ID'], axis=1, inplace=True)
df.drop(['B_ID'], axis=1, inplace=True)
df.drop(['R_Name'], axis=1, inplace=True)
df.drop(['B_Name'], axis=1, inplace=True)
df.drop(['Event_ID'], axis=1, inplace=True)
df.drop(['Fight_ID'], axis=1, inplace=True)

In [31]:
# combine less frequent values
percantage = 0.3
categorical = df.select_dtypes(include="object").columns
for column in categorical:
    if column == 'winner': continue
    values = df[column].value_counts().values
    qunatile = np.quantile(values, percantage)
    grouped = df.groupby(column)[column].transform('size')
    df.loc[grouped <= qunatile, column] = 'other'
    dummies =  pd.get_dummies(df[column])
    df.drop(column,axis=1, inplace=True)
    df = pd.concat([df, dummies], axis=1)


In [32]:
# fill with median to handle outliers
def fill_discrete_nan(data, column):
    data[column].fillna(data[column].median())
    return data

for column in df:
    if('Round' in column):
        df = fill_discrete_nan(df, column)

In [33]:
def fill_cont_nan(data, column):
    data[column].fillna(data[column].mean())
    return data

df = fill_cont_nan(df, 'B_Height')
df = fill_cont_nan(df, 'B_Weight')

df = fill_cont_nan(df, 'R_Height')
df = fill_cont_nan(df, 'R_Weight')

In [34]:
df = df.fillna(method='bfill')
df = df.fillna(method='bfill')

In [35]:
weightDf = df[["B_Weight", "R_Weight"]]
min_weight = weightDf.min().min()
max_weight = weightDf.max().max()
bins = np.linspace(min_weight, max_weight, num=5)
df['B_Weight'] = pd.cut(df["B_Weight"], bins=bins)
df['R_Weight'] = pd.cut(df["R_Weight"], bins=bins)

In [36]:
train, test = train_test_split(df, test_size=0.2)
train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)